In [ ]:
import pandas as pd

#Read in Census Data files from GoogleDrive
#To reproduce code replace with your own file path
files = ["/content/drive/MyDrive/project/meansOfTravel.csv",
         "/content/drive/MyDrive/project/journeyTime.csv",
         "/content/drive/MyDrive/project/educationLevel.csv",
         "/content/drive/MyDrive/project/employmentRate.csv",
         "/content/drive/MyDrive/project/centralHeating.csv",
         "/content/drive/MyDrive/project/waterSupply.csv",
         "/content/drive/MyDrive/project/sewerageFacility.csv",
         "/content/drive/MyDrive/project/renewableEnergy.csv"
]


columns_list = []
for file in files:
    df = pd.read_csv(file, nrows=0)
    columns_list.append(set(df.columns))

#Inspected list of columns common to all files
common_columns = set.intersection(*columns_list)
print(common_columns)


{'CSO_LEA', 'NUTS1_NAME', 'NUTS2', 'Reference_Geography', 'LOCAL_AUTHORITY', 'NUTS3_NAME', 'ED_ENGLISH', 'BUA_CODE', 'COUNTY', 'LA_CODE', 'NUTS1', 'Total dwellings (Normalisation)', 'Area unit of measure', 'GEOGDESC', 'GUID.1', 'Other vacant dwellings (Normalisation)', 'BUA_GUID', 'Temporarily absent (Normalisation)', 'Occupied (Normalisation)', 'LEA_OFFICIAL', 'LEA_ID', 'NUTS2_NAME', 'Shape__Area', 'LA_GUID', 'NUTS3', 'AREA', 'BUA_NAME', 'Total - Males (Normalisation)', 'ED_OFFICIAL', 'GEOGID', 'GUID', 'ED_GAEILGE', 'LEA_GUID', 'Unoccupied holiday homes (Normalisation)', 'Shape__Length', 'CONTAE', 'ObjectId', 'PROVINCE', 'Total Population (Normalisation)', 'Total - Females (Normalisation)', 'ED_ID_STR', 'SA_PUB2022', 'SA_GEOGID_2022', 'ED_GUID'}


In [ ]:
#Read in Means of Travel column names
meansOfTravelCols = set(pd.read_csv(files[0], nrows=0).columns)
#Removed common columns
uniqueMeansOfTravelCols = meansOfTravelCols - common_columns

#Analysed columns unique to Means of Travel data to determine feature extraction
uniqueMeansOfTravelCols

{'Bicycle - School, college or childcare',
 'Bicycle - Total',
 'Bicycle - Work',
 'Bus, minibus or coach - School, college or childcare',
 'Bus, minibus or coach - Total',
 'Bus, minibus or coach - Work',
 'Car driver - School, college or childcare',
 'Car driver - Total',
 'Car driver - Work',
 'Car passenger - School, college or childcare',
 'Car passenger - Total',
 'Car passenger - Work',
 'Motorcycle or scooter - School, college or childcare',
 'Motorcycle or scooter - Total',
 'Motorcycle or scooter - Work',
 'Not stated - School, college or childcare',
 'Not stated - Total',
 'Not stated - Work',
 'On foot - School, college or childcare',
 'On foot - Total',
 'On foot - Work',
 'Other (incl. lorry) - School, college or childcare',
 'Other (incl. lorry) - Total',
 'Other (incl. lorry) - Work',
 'Total',
 'Total - School, college or childcare',
 'Total - Work',
 'Train, DART or LUAS - School, college or childcare',
 'Train, DART or LUAS - Total',
 'Train, DART or LUAS - Work',
 '

In [ ]:
#Read in Means of Travel data
meansOfTravel = pd.read_csv(files[0])

#Created new column groupings for features:
col_total = 'Total'
col_not_stated = 'Not stated - Total'

#Personal vehicle modes of transport
personal_vehicle_cols = [
    'Motorcycle or scooter - Total',
    'Car driver - Total',
    'Car passenger - Total',
    'Van - Total'
]

#Walking and Cycling
walk_cycle_cols = [
    'On foot - Total',
    'Bicycle - Total'
]

#Public Transportation methods (Bus, train, tram)
public_transport_cols = [
    'Bus, minibus or coach - Total',
    'Train, DART or LUAS - Total'
]

#For calculating proportions, created denominator of total population who responded to question
meansOfTravel['denominator'] = meansOfTravel[col_total] - meansOfTravel[col_not_stated]

#Calculated proportion who travel to work/school via personal vehicle (Motorbike, Van, Car)
meansOfTravel['cleanPersonalVehicle'] = (
    meansOfTravel[personal_vehicle_cols].sum(axis=1) /
    meansOfTravel['denominator']
)

#Calculated proportion who travel to work/school by walking or cycling
meansOfTravel['cleanWalkCycle'] = (
    meansOfTravel[walk_cycle_cols].sum(axis=1) /
    meansOfTravel['denominator']
)

#Calculated proportion who travel to work/school by public transport (bus/tram/train)
meansOfTravel['cleanPublicTransport'] = (
    meansOfTravel[public_transport_cols].sum(axis=1) /
    meansOfTravel['denominator']
)

#Removed denominator column
meansOfTravel.drop(columns=['denominator'], inplace=True)

#Inspected new feature values
print(meansOfTravel[['cleanPersonalVehicle', 'cleanWalkCycle', 'cleanPublicTransport']].head())


   cleanPersonalVehicle  cleanWalkCycle  cleanPublicTransport
0              0.085714        0.533333              0.266667
1              0.413534        0.150376              0.300752
2              0.205357        0.464286              0.196429
3              0.137255        0.509804              0.300654
4              0.130435        0.565217              0.217391


In [ ]:
#Read in Journey Time column names
journeyTimeCols = set(pd.read_csv(files[1], nrows=0).columns)

#Removed common columns
uniqueJourneyTimeCols = journeyTimeCols - common_columns

#Analysed columns unique to Journey Time data to determine feature extraction
uniqueJourneyTimeCols

{'1 1/2 hours and over',
 '1 hour - under 1 1/2 hours',
 '1/2 hour - under 3/4 hour',
 '1/4 hour - under 1/2 hour',
 '3/4 hour - under 1 hour',
 'Not stated',
 'Total',
 'Under 15 mins'}

In [ ]:
#Read in Journey Time Data
journeyTime = pd.read_csv(files[1])

#Calculated population density metric (people per square meter) by dividing population by area
journeyTime['cleanPopulationDensity'] = journeyTime['Total Population (Normalisation)'] / journeyTime['Shape__Area']

In [ ]:

#Created new column groupings for features:

#Time categories below 30 mins
under_half_cols = [
    'Under 15 mins',
    '1/4 hour - under 1/2 hour'
]

#Time categories between 30-60 mins
under_hour_cols = [
    '1/2 hour - under 3/4 hour',
    '3/4 hour - under 1 hour'
]

#Time categories over 60 mins
over_hour_cols = [
    '1 hour - under 1 1/2 hours',
    '1 1/2 hours and over'
]

#For calculating proportions, created denominator of total population who responded to question
journeyTime['denom'] = journeyTime['Total'] - journeyTime['Not stated']

#Calculated proportion of population who travel to work/school in under 30 minutes
journeyTime['cleanUnderHalfHourJourney'] = journeyTime[under_half_cols].sum(axis=1) / journeyTime['denom']

#Calculated proportion of population who travel to work/school in 30-60 minutes
journeyTime['cleanUnderHourJourney'] = journeyTime[under_hour_cols].sum(axis=1) / journeyTime['denom']

#Calculated proportion of population who travel to work/school in over 60 minutes
journeyTime['cleanOverHourJourney'] = journeyTime[over_hour_cols].sum(axis=1) / journeyTime['denom']

#Removed denominator column
journeyTime.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(journeyTime[['cleanUnderHalfHourJourney', 'cleanUnderHourJourney', 'cleanUnderHourJourney']].head())


   cleanUnderHalfHourJourney  cleanUnderHourJourney  cleanUnderHourJourney
0                   0.438202               0.483146               0.483146
1                   0.630631               0.288288               0.288288
2                   0.659341               0.285714               0.285714
3                   0.424000               0.512000               0.512000
4                   0.714286               0.178571               0.178571


In [ ]:
#Read in Employment Rate column names
employmentRateCols = set(pd.read_csv(files[3], nrows=0).columns)

#Removed common columns
uniqueEmploymentRateCols = employmentRateCols - common_columns

#Analysed columns unique to Employment Rate data to determine feature extraction
uniqueEmploymentRateCols

{'At work - Females',
 'At work - Males',
 'At work - Total',
 'Long Term Unemployed - Females',
 'Long Term Unemployed - Males',
 'Long Term Unemployed - Total',
 'Looking after home/family - Females',
 'Looking after home/family - Males',
 'Looking after home/family - Total',
 'Looking for first regular job - Females',
 'Looking for first regular job - Males',
 'Looking for first regular job - Total',
 'Other - Females',
 'Other - Males',
 'Other - Total',
 'Retired - Females',
 'Retired - Males',
 'Retired - Total',
 'Student - Females',
 'Student - Males',
 'Student - Total',
 'Total',
 'Total - Females',
 'Total - Males',
 'Unable to work due to permanent sickness or disability - Females',
 'Unable to work due to permanent sickness or disability - Males',
 'Unable to work due to permanent sickness or disability - Total',
 'short Term Unemployed  - Females',
 'short Term Unemployed  - Males',
 'short Term Unemployed  - Total '}

In [ ]:
#Read in Employment Rate Data
employmentRate = pd.read_csv(files[3])

#Calculated employment rate by dividing the number of people at work by the total population
#N.B. There was no "Not-stated" field for this data
employmentRate['cleanEmploymentRate'] = (
    employmentRate['At work - Total'] /
    employmentRate['Total']
)

#Inspected new feature values
print(employmentRate[['cleanEmploymentRate']].head())


   cleanEmploymentRate
0             0.743590
1             0.764151
2             0.614286
3             0.664336
4             0.627586


In [ ]:
#Read in Education Level column names
educationLevelCols = set(pd.read_csv(files[2], nrows=0).columns)

#Removed common columns
uniqueEducationLevelCols = educationLevelCols - common_columns

#Analysed columns unique to Education Level data to determine feature extraction
uniqueEducationLevelCols

{'Advanced certificate/Completed apprenticeship - Females',
 'Advanced certificate/Completed apprenticeship - Males',
 'Advanced certificate/Completed apprenticeship - Total',
 'Doctorate(Ph.D) or higher - Females',
 'Doctorate(Ph.D) or higher - Males',
 'Doctorate(Ph.D) or higher - Total',
 'Higher certificate - Females',
 'Higher certificate - Males',
 'Higher certificate - Total',
 'Honours bachelor degree, professional qualification or both - Females',
 'Honours bachelor degree, professional qualification or both - Males',
 'Honours bachelor degree, professional qualification or both - Total',
 'Lower secondary - Females',
 'Lower secondary - Males',
 'Lower secondary - Total',
 'No formal education - Females',
 'No formal education - Males',
 'No formal education - Total',
 'Not stated - Females',
 'Not stated - Males',
 'Not stated - Total',
 'Ordinary bachelor degree or national diploma - Females',
 'Ordinary bachelor degree or national diploma - Males',
 'Ordinary bachelor degr

In [ ]:
#Read in Education Level data
educationLevel = pd.read_csv(files[2])

#Created new column groupings for features:

#Education levels below Leaving Certificate (Below Level 6)
no_lc_cols = [
    'No formal education - Total',
    'Primary education - Total',
    'Lower secondary - Total'
]

#Further education levels (Level 7 and above)
college_cols = [
    'Ordinary bachelor degree or national diploma - Total',
    'Honours bachelor degree, professional qualification or both - Total',
    'Postgraduate diploma or degree - Total',
    'Doctorate(Ph.D) or higher - Total'
]

#For calculating proportions, created denominator of total population who responded to question
educationLevel['denom'] = educationLevel['Total'] - educationLevel['Not stated - Total']

#Calculated proportion of population who do not have a Leaving Certificate qualification (Level 6)
educationLevel['cleanNoLC'] = educationLevel[no_lc_cols].sum(axis=1) / educationLevel['denom']

#Calculated proportion of population who have a third level education qualification (Level 7 or higher)
educationLevel['cleanCollegeEducation'] = educationLevel[college_cols].sum(axis=1) / educationLevel['denom']

#Removed denominator column
educationLevel.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(educationLevel[['cleanNoLC', 'cleanCollegeEducation']].head())


   cleanNoLC  cleanCollegeEducation
0   0.053097               0.716814
1   0.205674               0.397163
2   0.033333               0.788889
3   0.194915               0.440678
4   0.067568               0.689189


In [ ]:
#Read in Central Heating Column Names
centralHeatingCols = set(pd.read_csv(files[4], nrows=0).columns)

#Removed common columns
uniqueCentralHeatingCols = centralHeatingCols - common_columns

#Analysed columns unique to Central Heating data to determine feature extraction
uniqueCentralHeatingCols

{'Coal (incl. anthracite)',
 'Electricity',
 'Liquid petroleum gas (LPG)',
 'Natural gas',
 'No central heating',
 'Not stated',
 'Oil',
 'Other',
 'Peat (incl. turf)',
 'Total',
 'Wood (incl. wood pellets)'}

In [ ]:
#Read in Central Heating data
centralHeating = pd.read_csv(files[4])

#For calculating proportions, created denominator of total population who responded to question
centralHeating['denom'] = centralHeating['Total'] - centralHeating['Not stated']

#Calculated proportion of population who have no central heating
centralHeating['cleanNoCentralHeating'] = (
    centralHeating['No central heating'] / centralHeating['denom']
)

#Removed denominator column
centralHeating.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(centralHeating[['cleanNoCentralHeating']].head())


   cleanNoCentralHeating
0               0.029851
1               0.020619
2               0.000000
3               0.038462
4               0.037736


In [ ]:
#Read in Water Supply Columns
waterSupplyCols = set(pd.read_csv(files[5], nrows=0).columns)

#Removed common columns
uniqueWaterSupplyCols = waterSupplyCols - common_columns

#Analysed columns unique to Water Supply data to determine feature extraction
uniqueWaterSupplyCols

{'Group scheme with private source',
 'Group scheme with public source',
 'None',
 'Not stated',
 'Other private source',
 'Public main',
 'Total'}

In [ ]:
#Read in Water Supply data
waterSupply = pd.read_csv(files[5])

#For calculating proportions, created denominator of total population who responded to question
waterSupply['denom'] = waterSupply['Total'] - waterSupply['Not stated']

#Created new column groupings for features:

#Public water sources
public_cols = [
    'Public main',
    'Group scheme with public source'
]

#Calculated proportion of population who access water through the public system
waterSupply['cleanServedByPublicSystem_Water'] = waterSupply[public_cols].sum(axis=1) / waterSupply['denom']

#Removed denominator column
waterSupply.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(waterSupply[['cleanServedByPublicSystem_Water']].head())


   cleanServedByPublicSystem_Water
0                         0.988235
1                         0.964602
2                         0.985714
3                         0.967742
4                         1.000000


In [ ]:
#Read in Sewerage Facility column names
sewerageFacilityCols = set(pd.read_csv(files[6], nrows=0).columns)

#Removed common columns
uniqueSewerageFacilityCols = sewerageFacilityCols - common_columns

#Analysed columns unique to Sewerage Facility data to determine feature extraction
uniqueSewerageFacilityCols

{'Individual septic tank',
 'No sewerage facility',
 'Not stated',
 'Other',
 'Other individual treatment',
 'Public scheme',
 'Total'}

In [ ]:
#Read in Sewerage Facility Data
sewerageFacility = pd.read_csv(files[6])

#For calculating proportions, created denominator of total population who responded to question
sewerageFacility['denom'] = sewerageFacility['Total'] - sewerageFacility['Not stated'] - sewerageFacility['Other']

#Calculated proportion of population who access sewerage facilities from the public system
sewerageFacility['cleanServedByPublicSystem_Sewerage'] = sewerageFacility['Public scheme'] / sewerageFacility['denom']

#Removed denominator column
sewerageFacility.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(sewerageFacility[['cleanServedByPublicSystem_Sewerage']].head())


   cleanServedByPublicSystem_Sewerage
0                            0.987952
1                            0.963964
2                            0.985294
3                            0.916667
4                            0.942857


In [ ]:
#Read in Renewable Energy columns names
renewableEnergyCols = set(pd.read_csv(files[7], nrows=0).columns)

#Removed common columns
uniqueRenewableEnergyCols = renewableEnergyCols - common_columns

#Analysed columns unique to Renewable Energy data to determine feature extraction
uniqueRenewableEnergyCols

{'Has renewable Energy',
 'No Renewable Energy',
 'Not stated',
 'Total Renewables'}

In [ ]:
#Read in Renewable Energy data
renewableEnergy = pd.read_csv(files[7])

#For calculating proportions, created denominator of total population who responded to question
renewableEnergy['denom'] = renewableEnergy['Total Renewables'] - renewableEnergy['Not stated']

#Calculated proportion of local population who have renewable energy sources
renewableEnergy['cleanHasRenewableEnergy'] = renewableEnergy['Has renewable Energy'] / renewableEnergy['denom']

#Removed denominator column
renewableEnergy.drop(columns=['denom'], inplace=True)

#Inspected new feature values
print(renewableEnergy[['cleanHasRenewableEnergy']].head())


   cleanHasRenewableEnergy
0                 0.000000
1                 0.000000
2                 0.064516
3                 0.019608
4                 0.023810


In [ ]:
#Selected columns to retain from those common to all dataframes
core_cols = [
    "GUID",
    "GEOGID",
    "ED_ENGLISH",
    "LOCAL_AUTHORITY",
    "Total Population (Normalisation)",
    "Total dwellings (Normalisation)",
    "Shape__Area"
]

#Reduced each dataframe to core columns + new engineered features
df1 = meansOfTravel[core_cols + [c for c in meansOfTravel.columns if c.startswith("clean")]]
df2 = journeyTime[core_cols + [c for c in journeyTime.columns if c.startswith("clean")]]
df3 = educationLevel[core_cols + [c for c in educationLevel.columns if c.startswith("clean")]]
df4 = employmentRate[core_cols + [c for c in employmentRate.columns if c.startswith("clean")]]
df5 = centralHeating[core_cols + [c for c in centralHeating.columns if c.startswith("clean")]]
df6 = waterSupply[core_cols + [c for c in waterSupply.columns if c.startswith("clean")]]
df7 = sewerageFacility[core_cols + [c for c in sewerageFacility.columns if c.startswith("clean")]]
df8 = renewableEnergy[core_cols + [c for c in renewableEnergy.columns if c.startswith("clean")]]

from functools import reduce

#Merged files into one dataframe on core columns
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]

cleaned = reduce(
    lambda left, right: pd.merge(left, right, on=core_cols, how="left"),
    dfs
)

#Removed 'clean' prefix from feature columns
cleaned.columns = cleaned.columns.str.replace(r"^clean", "", regex=True)

#Saved cleaned data to Csv
#Data now contains one row per small area with info from all original Census datasets
cleaned.to_csv("/content/drive/MyDrive/project/censusDataCleaned.csv", index=False)

#Inspected new list of columns
print(cleaned.columns)


Index(['GUID', 'GEOGID', 'ED_ENGLISH', 'LOCAL_AUTHORITY',
       'Total Population (Normalisation)', 'Total dwellings (Normalisation)',
       'Shape__Area', 'PersonalVehicle', 'WalkCycle', 'PublicTransport',
       'PopulationDensity', 'UnderHalfHourJourney', 'UnderHourJourney',
       'OverHourJourney', 'NoLC', 'CollegeEducation', 'EmploymentRate',
       'NoCentralHeating', 'ServedByPublicSystem_Water',
       'ServedByPublicSystem_Sewerage', 'HasRenewableEnergy'],
      dtype='object')
